![aiedge](https://ai-edge.be/assets/img/ai-edge.fe085446.png)

# CAR CLASSIFIER
This notebook will guide you through the process of testing a pretrained MobileNet car/no-car cassification network and exporting it from TensorFlow to TFLite Micro. In addition to that, we also quantize the model to 8-bits.
Afterwards, we can run this model on a micro-controller, such as the ESP32.

For more information, check out the TensorFlow documentation: https://www.tensorflow.org/lite.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

print(tf.__version__)

2.4.1


## Loading the Model

Here we will load a model which includes architecture and weight data.  
This model has been pretrained on the coco dataset and is stored in the **model/** directory.

In [2]:
model = tf.keras.models.load_model('./model', compile=True)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
Data_augmentation (Sequentia (None, 96, 96, 3)         0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 96, 96, 3)         0         
_________________________________________________________________
model_1 (Functional)         (None, 1)                 218801    
Total params: 218,801
Trainable params: 213,329
Non-trainable params: 5,472
_________________________________________________________________


## Testing the Model
Before we quantize and convert the model to TFLite, we first want to run the original tensorflow model through a test dataset.  
These results can then be compared with the results from our quantized model on the ESP-EYE, in order to validate our model is working correctly.

### Testing Dataset
We supplied a testing dataset in the **dataset/** folder.  
The folder contains two subfolders **car/** and **no_car/**, which contain images with and without cars respectively.

In [3]:
test_dataset = image_dataset_from_directory(
    './dataset',
    class_names =('no_car', 'car'),
    shuffle=True,
    batch_size=1,
    image_size=model.input_shape[1:3],
    interpolation='bilinear',
)

Found 100 files belonging to 2 classes.


### Testing Accuracy
We can now use this dataset to evaluate our model.

In [4]:
loss, accuracy = model.evaluate(test_dataset)

print(f'Test Accuracy = {100 * accuracy:.2f}%')
print(f'Test loss = {loss:.2f}')

100/100 [==============================] - 2s 11ms/step - loss: 0.2712 - accuracy: 0.9100
Test Accuracy = 91.00%
Test loss = 0.27


### Visualize Test Examples
We can also use this dataset to visualize some results.

In [5]:
# Create a matplotlib plot
plt.figure(figsize=(10, 10))

# Loop through 9 images
for i, (images, labels) in enumerate(test_dataset.take(9)):
    
    # Create a 3x3 grid of images and select image I
    ax = plt.subplot(3, 3, i + 1)
    
    # Plot the image
    plt.imshow(images[0].numpy().astype('uint8'))
    
    # Perform a prediction
    predicted_probability = model.predict(images[0:1])[0]
    
    # Interpret the probability into the correct class 'car' or 'no_car'
    predicted_class = 'car' if predicted_probability > 0.5 else 'no_car'
    
    # Add ground truth and prediction as a title
    plt.title(
        f'Ground Truth = {test_dataset.class_names[labels[0]]}\n'
        f'Prediction = {predicted_class}'
    )
    
    # Disable plotting axis
    plt.axis('off')

## Exporting the Model
Now we need to export the model to TFLite.  
This time, we will also use TFLite to quantize the model to int8.

### Workaround a bug

Since Tensorflow 2.3 there is [a bug](https://github.com/tensorflow/tensorflow/issues/45256) with the reshape layer when this layer is converted to tensorflow lite.  
For some reason the converter produces a non supported sequence of `Shape → Strided Slice → Pack → Reshape` for each reshape layer instead of just using the shape layer.
Unfortunately, earlier Tensorflow versions that do not have this bug, have limited INT8 support which we want to use.

We work around this bug by removing the reshape layer in our tensorflow model.  
The only disadvantage is that the output will have a shape of `(None, 1, 1, 1)` instead of `(None, 1)`, so we need to take this into consideration.

Note that this fix removes the preprocessing layers too which would be removed by the converter anyway if kept.

In [6]:
# Select only the model part
model = model.layers[-1]

# Select the output of the layer just before the reshape layer
x = model.layers[-3].output

# Add an activation function on top
x = tf.keras.layers.Activation(activation='sigmoid', name='predictions')(x)

# Compose the new model
model = tf.keras.Model(model.input, x)

# Print model summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenet_0.25_224 (Function (None, 3, 3, 256)         218544    
_________________________________________________________________
average_pooling2d_6 (Average (None, 1, 1, 256)         0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 256)         0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 1)           257       
_________________________________________________________________
predictions (Activation)     (None, 1, 1, 1)           0         
Total params: 218,801
Trainable params: 213,329
Non-trainable params: 5,472
___________________________________________________

### Quantized TFLite Model
In addition to the the standard conversion, we are also going tell the Tensorflow Lite converter to apply Full integer quantization.  
_You can use the [following guide](https://www.tensorflow.org/lite/performance/post_training_quantization#integer_only) to fill in the blanks in the next code section._

In [7]:
# Create a representative dataset for quantization
def representative_dataset():
    for images, _ in test_dataset.take(100):
        for image in images:
            res = np.expand_dims(image.numpy() / 255., axis=0).astype(np.float32)
            yield [res]

# Create the TFLite converter from our Keras model (not from saved model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable full integer quantization
converter.representative_dataset = representative_dataset
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8 
converter.inference_output_type = tf.int8 

# Convert our model
tflite_model = converter.convert()

# Save tflite model to a file
with open('carclassifier.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7zs1wq27/assets


We have now saved our TFLite model to a file, which we can visualize with Netron.  
<a href="http://localhost:1234" target="_blank">Click here</a> to open netron and select your file to visualize it.  
After clicking on "Open Model...", you can press **CTRL+L** and enter the following path: _~/aiedge/carclassifier_hard/notebook/carclassifier.tflite_

### Convert to C++ data
Finally, we need to convert the TFLite model to a C++ array.

In [8]:
!xxd -i carclassifier.tflite > carclassifier.cpp
!cat carclassifier.cpp

unsigned char carclassifier_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x12, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x08, 0x00, 0x00, 0x00, 0x04, 0x00, 0x12, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x00, 0x00, 0x48, 0x00, 0x00, 0x00, 0xa8, 0x5b, 0x03, 0x00,
  0xb8, 0x5b, 0x03, 0x00, 0xcc, 0x02, 0x05, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x08, 0x00, 0x00, 0x00, 0x58, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x6f, 0x6e, 0x00, 0x59, 0x00, 0x00, 0x00, 0x5c, 0x5b, 0x03, 0x00,
  0x50, 0x5b, 0x03, 0x00, 0x20, 0x5b, 0x03, 0x00, 0x08, 0x59, 0x03, 0x00,
  0xf8, 0x56, 0x03, 0x00, 0xe8, 0x54, 0x03, 0x00, 0xd8, 0x50, 0x03, 0x00,
  0xa8, 0x50, 0x03, 0x00, 0x58, 0x50, 0x03, 0x00, 0xc8, 0x4f, 0x03, 0x0

  0x04, 0x0c, 0x0b, 0xec, 0x1d, 0x19, 0x0c, 0x17, 0x15, 0x20, 0x0f, 0x0d,
  0xdd, 0x20, 0x19, 0x3a, 0x1d, 0xfb, 0x19, 0x05, 0xe6, 0x26, 0xd7, 0x0e,
  0xd1, 0x07, 0x42, 0xef, 0x16, 0xf4, 0x13, 0xdd, 0x23, 0xe9, 0x45, 0x13,
  0x09, 0x04, 0x11, 0x10, 0x19, 0x3f, 0xe3, 0x3e, 0x04, 0xfe, 0x18, 0x00,
  0xd2, 0x0b, 0xed, 0x02, 0x1d, 0x0f, 0x1f, 0x21, 0xde, 0x08, 0xe9, 0x01,
  0xe6, 0xf0, 0xed, 0xef, 0xe8, 0x20, 0x0a, 0x25, 0x24, 0xec, 0xd5, 0x09,
  0x20, 0x01, 0x28, 0xbd, 0xda, 0xf5, 0x19, 0xee, 0xed, 0x36, 0x11, 0x34,
  0x0b, 0x00, 0x02, 0xff, 0x24, 0x02, 0x10, 0xdc, 0xda, 0xfd, 0x2e, 0x25,
  0xf2, 0xd9, 0xf4, 0x2d, 0x08, 0x3f, 0x20, 0xf6, 0x1e, 0xe4, 0xfa, 0xd9,
  0xce, 0xe7, 0xe9, 0x07, 0xef, 0xcc, 0xcd, 0xd5, 0xee, 0x1d, 0xed, 0x1e,
  0x09, 0xee, 0x13, 0xdf, 0xe6, 0xcc, 0x39, 0xf1, 0x20, 0x11, 0xeb, 0xf7,
  0x22, 0x04, 0xf9, 0x14, 0x09, 0x1d, 0x1c, 0xf8, 0x12, 0xf4, 0x05, 0x1e,
  0x16, 0xfc, 0xe6, 0x0d, 0x05, 0x11, 0xf5, 0xca, 0xfd, 0x14, 0x2c, 0x00,
  0x16, 0x16, 0x10, 0x57, 0xf8, 0x0f, 

  0x10, 0x16, 0x00, 0xfe, 0x1d, 0xfa, 0x16, 0xe6, 0xf9, 0x02, 0x19, 0x03,
  0x0b, 0xfa, 0xf5, 0xf6, 0xf3, 0x14, 0x04, 0x0f, 0xdf, 0xde, 0x02, 0x0e,
  0xf9, 0xe3, 0xee, 0x09, 0x02, 0xf3, 0xfd, 0x11, 0x0d, 0xf7, 0x03, 0x01,
  0x03, 0xee, 0xf5, 0xf2, 0x07, 0xe4, 0xcb, 0xed, 0x12, 0xfe, 0x02, 0xf0,
  0xea, 0xf9, 0xfc, 0x24, 0xf2, 0xe8, 0xf6, 0xe6, 0x01, 0xf7, 0xf6, 0x02,
  0x0e, 0xd8, 0xeb, 0xd1, 0xf2, 0xef, 0xf7, 0xf3, 0x7f, 0x08, 0x00, 0xf2,
  0x06, 0xfd, 0x06, 0xdd, 0xfc, 0xfb, 0x00, 0x1e, 0xf1, 0x05, 0x08, 0x19,
  0x08, 0x00, 0x01, 0xec, 0xdb, 0xfb, 0xd5, 0x75, 0x9a, 0xb8, 0x15, 0x29,
  0xc1, 0xff, 0x17, 0x00, 0x01, 0xf9, 0x12, 0xc4, 0x01, 0x0a, 0x24, 0xdc,
  0xd7, 0xb4, 0xfa, 0xe6, 0x24, 0x1b, 0xe6, 0x28, 0xc2, 0xfe, 0xc3, 0xb5,
  0xe6, 0x46, 0x04, 0xee, 0x02, 0x06, 0xfd, 0xf3, 0x27, 0x47, 0x15, 0x13,
  0xf8, 0x16, 0xc8, 0x07, 0x1d, 0x0a, 0x30, 0x18, 0xca, 0x3c, 0xbf, 0xe4,
  0xf9, 0x2a, 0x08, 0xfc, 0x97, 0x2c, 0xe5, 0x0e, 0xe8, 0xcb, 0xf2, 0xcf,
  0xf7, 0xe5, 0xab, 0x01, 0xe8, 0x10, 

  0x2b, 0xaa, 0xff, 0xff, 0x57, 0xec, 0xff, 0xff, 0x43, 0xcf, 0xff, 0xff,
  0x6d, 0xd9, 0x00, 0x00, 0xfa, 0xf8, 0xff, 0xff, 0xb5, 0xec, 0xff, 0xff,
  0x1f, 0x78, 0x00, 0x00, 0x87, 0xdd, 0xff, 0xff, 0x23, 0xfe, 0xff, 0xff,
  0x91, 0xf5, 0xff, 0xff, 0x4f, 0xe0, 0xff, 0xff, 0x75, 0x7b, 0x00, 0x00,
  0xbc, 0xfa, 0xff, 0xff, 0xf3, 0xaa, 0x00, 0x00, 0x3b, 0xf7, 0xff, 0xff,
  0x62, 0x04, 0x00, 0x00, 0xef, 0x7d, 0x00, 0x00, 0x0b, 0xfd, 0xff, 0xff,
  0xf2, 0xfd, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00,
  0xa3, 0xed, 0xff, 0xff, 0x42, 0x90, 0x00, 0x00, 0x6d, 0xb1, 0x01, 0x00,
  0xbf, 0x5e, 0x00, 0x00, 0xbe, 0xcd, 0xff, 0xff, 0xbd, 0xf5, 0xff, 0xff,
  0xd1, 0xc5, 0xff, 0xff, 0x47, 0x85, 0xff, 0xff, 0xb4, 0x1d, 0x00, 0x00,
  0x5d, 0xc9, 0x00, 0x00, 0x12, 0x1a, 0x01, 0x00, 0x60, 0x7d, 0x00, 0x00,
  0xa1, 0x85, 0x00, 0x00, 0x62, 0x82, 0x00, 0x00, 0x7a, 0xc4, 0xff, 0xff,
  0x42, 0xe7, 0xff, 0xff, 0x37, 0xf2, 0xff, 0xff, 0x01, 0xf6, 0xff, 0xff,
  0x4c, 0x53, 0x00, 0x00, 0x31, 0xdd, 

  0x77, 0x91, 0x49, 0x38, 0x95, 0x8c, 0x29, 0x39, 0x49, 0x04, 0xdf, 0x38,
  0xb5, 0x29, 0xab, 0x38, 0xde, 0x88, 0x8a, 0x38, 0xcd, 0x69, 0xe7, 0x38,
  0x2c, 0x6a, 0x83, 0x38, 0xf3, 0x18, 0xbe, 0x38, 0x76, 0xe4, 0xfa, 0x38,
  0x64, 0xb7, 0xc7, 0x38, 0xf0, 0xc9, 0x1e, 0x38, 0x06, 0x07, 0x3f, 0x39,
  0x2f, 0x84, 0x2a, 0x39, 0x27, 0x39, 0x70, 0x38, 0xa7, 0x71, 0x11, 0x39,
  0x85, 0xfd, 0x4d, 0x39, 0x97, 0x15, 0x46, 0x38, 0x9e, 0x03, 0x8f, 0x38,
  0xd0, 0x9f, 0xc1, 0x38, 0xa7, 0xed, 0xe5, 0x38, 0x0f, 0xcd, 0xd9, 0x38,
  0x79, 0x5e, 0xbe, 0x38, 0x2d, 0xac, 0x84, 0x39, 0xde, 0xd9, 0xc7, 0x38,
  0x5e, 0xeb, 0x68, 0x38, 0x9d, 0x3f, 0x21, 0x38, 0x31, 0xdd, 0x00, 0x38,
  0xd2, 0xa9, 0xe6, 0x38, 0x41, 0xd4, 0x6d, 0x39, 0xd0, 0xc5, 0x22, 0x39,
  0x37, 0x25, 0xf5, 0x38, 0xa1, 0x13, 0xa1, 0x38, 0x14, 0x48, 0xcd, 0x38,
  0xc6, 0x0b, 0xcc, 0x38, 0x79, 0xe5, 0xd8, 0x38, 0x9d, 0x39, 0xbd, 0x38,
  0x3e, 0x15, 0x4c, 0x39, 0x30, 0xc7, 0xc0, 0x38, 0x20, 0x35, 0x84, 0x38,
  0x36, 0x00, 0x00, 0x00, 0x6d, 0x6f, 

## Compare Model Probabilities
As a basic sanity check, we will compare the output probabilities of our TensorFlow model with the quantized TFLite model.  
If all went well, the difference should be pretty small.

The advantage of comparing the outputs as opposed to looking at the testing accuracy, is that this code could theoretically be used for any network, regardless of its task.
We indeed simply look at the output tensors and compute the MSE between our floating point and quantized model.

_You can use the [following guide](https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python) to fill in the blanks in the next code section (scroll down to the second code block)._

In [9]:
# Load the TFLite model from the path './carclassifier.tflite'
tflite_model = tf.lite.Interpreter('./carclassifier.tflite')

# Allocate memory
tflite_model.allocate_tensors()

# Get input & output details
# These are necessary to know where we need to place the input in memory and where we get the output
input_details = tflite_model.get_input_details()
output_details = tflite_model.get_output_details()

# Arrays to store the output probabilities
tf_outputs = []
tflite_outputs = []

# Loop through 25 images
for image, label in test_dataset.take(25):
    
    # Convert the input image ([0,255] uint8) for our quantized model ([0,1] float32)
    tf_input = tf.cast(image / 255, tf.float32)
    
    # Predict using tensorflow model
    tf_output = model.predict(tf_input)
    
    # Append probability to list
    tf_outputs.append(tf_output)
    
    
    # Convert the input image ([0,255] uint8) for our quantized model ([-128,127] int8)
    # HINT: Use tf.cast()
    tflite_input = tf.cast(image - 128, tf.int8)

    # Predict using TFLite model
    tflite_model.set_tensor(input_details[0]['index'], tflite_input)
    tflite_model.invoke()
    tflite_output = tflite_model.get_tensor(output_details[0]['index'])

    # Convert the model output ([-128,127] int8) to a probability ([0,1] float32)
    tflite_output = (tf.cast(tflite_output, tf.float32) + 128) / 255

    # Append probability to list
    tflite_outputs.append(tflite_output)

    
# Compute the Mean Squared Error
tf_outputs = np.concatenate(tf_outputs)
tflite_outputs = np.concatenate(tflite_outputs)
mse = ((tf_outputs - tflite_outputs) ** 2).mean()

print(tf_outputs.shape, tflite_outputs.shape)
print("Mean squared error =", mse)

(25, 1, 1, 1) (25, 1, 1, 1)
Mean squared error = 0.0012812779


![eavise](https://gitlab.com/EAVISE/branding/logo/-/raw/master/twitter/header_500.png)